This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with a simple model, gain corruptions and a gain solver,
then again with a more complicated sky and beam model,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
gleamfile = "/data/EOS_1/mit183/gleamegc.dat"
eb_ms = "/data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms"
eb_coeffs = "/data/EOS_1/mit183/SKA/SP-4626/ska-sdp-func-everybeam/coeffs"

In [3]:
# Simple bandpass calibration of X and Y gains with a user-defined sky model
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-18 12:34:13,671 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-18 12:34:13,672 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-18 12:34:13,672 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-18 12:34:13,693 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-18 12:34:14,246 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-18 12:34:14,404 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-18 12:34:15,004 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-18 12:34:15,005 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-18 12:34:15,785 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-18 12:34:15,907 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12

pipeline ran in 4.4 seconds


In [4]:
# Simple bandpass calibration of X and Y gains with GLEAM and EveryBeam
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-18 12:34:18,127 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-18 12:34:18,128 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2024-12-18 12:34:18,129 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-18 12:34:18,130 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2024-12-18 12:34:18,130 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-18 12:34:18,130 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2024-12-18 12:34:18,131 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2024-12-18 12:34:18,725 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-18 12:34:18,726 - workflow.pipeline_config - INFO - LSM: found 37 components
2024-12-18 12:34:18,735 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-18 12:34:18,806 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-18 12:34:22,147 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-18 12:34:22,728 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-18 12:34:22,729 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-18 12:34:23,536 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-18 12:34:23,616 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2024-12-18 12:34:23,617 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-18 12:34:23,617 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2024-12-18 12:34:23,618 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2024-12-18 12:34:24,264 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-18 12:34:24,265 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2024-12-18 

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-18 12:34:28,356 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2024-12-18 12:34:28,375 - pipeline.bandpass_calibration - INFO - Checking results
2024-12-18 12:34:28,380 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 10.7 seconds


In [5]:
# Bandpass calibration with leakage
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
        "gains": True,
        "leakage": True,
        "solver": "normal_equations",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-18 12:34:28,785 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-18 12:34:28,786 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2024-12-18 12:34:28,787 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-18 12:34:28,788 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2024-12-18 12:34:28,789 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-18 12:34:28,789 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2024-12-18 12:34:28,790 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2024-12-18 12:34:29,380 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-18 12:34:29,381 - workflow.pipeline_config - INFO - LSM: found 37 components
2024-12-18 12:34:29,390 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-18 12:34:29,421 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-18 12:34:32,672 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-18 12:34:32,674 - workflow.utils - INFO - Applying direction-independent leakage corruptions
2024-12-18 12:34:33,240 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-18 12:34:33,241 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-18 12:34:34,164 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-18 12:34:34,232 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2024-12-18 12:34:34,233 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-18 12:34:34,234 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2024-12-18 12:34:34,235 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2024-12-18 12:34:34,914 - processing_tasks.lsm - INFO - extracted 37 GLEAM components


Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-18 12:34:40,386 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2024-12-18 12:34:40,401 - pipeline.bandpass_calibration - INFO - Checking results
2024-12-18 12:34:40,404 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 12.0 seconds
